# Data input/output

Chris Mutel ([PSI](https://www.psi.ch/)), Pascal Lesage ([CIRAIG](http://www.ciraig.org/en/))

## Learning objectives  
  - Learn how to input LCI data to Brightway in different ways:  
    - Programmatically, via dictionaries created directly in Python  
    - Excel/cvs importers  
    - SimaPro csv
      - Two unit-process example
      - Agribalyse
    - Importing ecoSpold, ecoSpold2

## Standard inputs and setup

In [1]:
import brightway2 as bw
import os

Setting the project

In [2]:
bw.projects.set_current('bw2_seminar_2017')

Assigning our database to a variable

In [6]:
eidb = bw.Database('ecoinvent 2.2')

## Context

Performing LCA generally requires:
  - Background LCI data (e.g. an LCI database)  
  - Foreground LCI data  
  - Sets of characterization factors.    
This section will deal with the way data is input to Brightway

Useful documentation about what a database in Brightway is can be found [here](http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/Databases.ipynb) and [here](https://docs.brightwaylca.org/intro.html#inventory-databases).

# Creating a database programmatically

One can create a database via a database. This database would include activities that would contain information about the activities themselves as well as information about the exchanges that are output to this activity (although these get seperated out when the data is written to the actual `database.db`). Let's look at the components of a database for a random activity:

In [7]:
random_act = eidb.random()

In [8]:
# Information about activities themselves:
random_act.as_dict()

{'authors': [{'address': 'Reckenholzstr.191, 8046 Zürich',
   'company': 'ART',
   'country': 'CH',
   'email': 'art@ecoinvent.org',
   'name': 'Sebastiano Meier'}],
 'categories': ['agricultural means of production', 'seed'],
 'code': '43dc021644399e95cfcd10fdfb94a27e',
 'comment': "Refers to 1 kg of rye seed (fresh weight), with a maximum water content of 15%\nThe seed produced at the farm is transported to the processing centre, treated (pre-cleaning, cleaning, eventually drying, chemical dressing (for integrated production) and bag filling), stored and afterwards transported to the regional storage centre. No data on wastewater production were available. \nTechnology:  Refers to average seed processing\nTime period:  Date of the major data sources.\nProduction volume:  Consumption of rye seed in Switzerland is 1'149 t/year, 62% imported\nSampling:  Data based on a literature, expert information and enquiry of seed processing plants.\nExtrapolations:  Production of imported seed is 

In [9]:
# Information associated with the exchanges of this activity:
[exc for exc in random_act.exchanges()][1].as_dict()

{'amount': 0.024,
 'categories': ('electricity', 'supply mix'),
 'comment': '(2,3,1,1,1,5)',
 'input': ('ecoinvent 2.2', '261fb14cc1671ff51389716f4bc0d7b2'),
 'loc': -3.7297014486341915,
 'location': 'CH',
 'name': 'electricity, low voltage, at grid',
 'negative': False,
 'output': ('ecoinvent 2.2', '43dc021644399e95cfcd10fdfb94a27e'),
 'scale': 0.09116077839697724,
 'type': 'technosphere',
 'uncertainty type': 2,
 'unit': 'kilowatt hour'}

Let's create one for our simple LCA:

In [10]:
database_as_dictionary = bw.Database("Database as dict")

water_bottle_data = {
    ("Database as dict", "Some code for the bottle production"): {
        "name": "Water bottle production",
        'unit': 'unit',
        'location': 'CH',
        'categories': ("Some made up", "category here"),
        "exchanges": [{
            "amount": 0.33,
            "input": ('ecoinvent 2.2', 'c028a331b2ce1cd30ce326c3ba284a62'), #Aluminium
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kg"},
                      {
            "amount": 0.33,
            "input": ('ecoinvent 2.2', 'ea5e562ab50994b90b7b3e0a3cd0e498'), #Deformation stroke
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kg"}
        ],
        },
    ("Database as dict", "Some code for drinking a bottle full of water"): {
        "name": "Water drinking",
        'unit': 'liter',
        'location': 'CH',
        'categories': ("Another made up", "category here"),
        "exchanges": [{
            "amount": 1,
            "input": ("Database as dict", "Some code for the bottle production"), #Our water bottle
            "type": "technosphere",
            "uncertainty type":5,
            "loc":0.005,
            "minimum":0.0005,
            "maximum":0.05,
            "unit": "kg"},
                      {
            "amount": 1,
            "input": ('ecoinvent 2.2', 'b80ba272db9a3202e07a29d7574cff80'), #Water
            "type": "technosphere",
            "uncertainty type":0,
            "unit=": "kg"}],
        }
}

In [11]:
database_as_dictionary.write(water_bottle_data)

Writing activities to SQLite3 database:
0%  100%
[##] | ETA: 00:00:00
Total time elapsed: 00:00:00


Title: Writing activities to SQLite3 database:
  Started: 03/29/2017 07:37:33
  Finished: 03/29/2017 07:37:33
  Total time elapsed: 00:00:00
  CPU %: 118.60
  Memory %: 2.06


In [14]:
len(database_as_dictionary)

2

In [15]:
drinking_act = [act for act in database_as_dictionary if "drinking" in act['name']][0]
drinking_act

'Water drinking' (liter, CH, ('Another made up', 'category here'))

In [16]:
lca = bw.LCA({drinking_act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))
lca.load_lci_data()

In [20]:
water_bottle_LCA_from_dict_input = bw.LCA({drinking_act:1}, ('IPCC 2013', 'climate change', 'GWP 100a'))

In [21]:
water_bottle_LCA_from_dict_input.lci()
water_bottle_LCA_from_dict_input.lcia()
water_bottle_LCA_from_dict_input.score

0.565134555796863

# Importing from CSV or Excel

## Choose: the easy way or the hard way

If you choose the easy way, just to the "easy way" section.

### The hard way

We will import a dataset in Excel, but this dataset has errors that we will have to fix.

In [43]:
imp = bw.ExcelImporter(os.path.join("files", "excel_importer_example_problems.xlsx"))
imp.apply_strategies()
imp.match_database(fields=('name', 'unit', 'location'))
imp.match_database("ecoinvent 2.2", fields=('name', 'unit', 'location'))
imp.statistics()

Extracted 3 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 14 strategies in 0.31 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
2 datasets
6 exchanges
0 unlinked exchanges
  


(2, 6, 0)

Oops... we have some problems. Let's export our in-process data to a Excel sheet and see what went wrong. It is probably also sensible to load our input Excel sheet at the same time.

In [41]:
imp.write_excel()

Wrote matching file to:
/Users/cmutel/Library/Application Support/Brightway3/bw2_seminar_2017.058cb6360ae51ecc25fe811e209c8b06/output/db-matching-BW2-Excel-water-bottle-import.xlsx


It looks like we have a few small inconsistencies, like the name of the processes or their locations. Try to fix these on your own, and re-run the code two cells above to get an idea of your progress. The attributes used for matching (`fields`) must match exactly, though they are case-insensitive.

If you can't get it to work, you can change the name of the import file to "excel_importer_example_fixed.xlsx"; this already has the necessary corrections.

If you need to find the ecoinvent processes, you can search ecoinvent (as you should already know :)

In [42]:
db = bw.Database("ecoinvent 2.2")
db.search("aluminium secondary")

['aluminium, secondary, from old scrap, at plant' (kilogram, RER, ['metals', 'extraction']),
 'aluminium, secondary, from new scrap, at plant' (kilogram, RER, ['metals', 'extraction']),
 'aluminium scrap, new, at plant' (kilogram, RER, ['metals', 'extraction']),
 'aluminium, production mix, at plant' (kilogram, RER, ['metals', 'extraction']),
 'aluminium, production mix, wrought alloy, at plant' (kilogram, RER, ['metals', 'extraction']),
 'aluminium, production mix, cast alloy, at plant' (kilogram, RER, ['metals', 'extraction'])]

### The easy way

Import a file where everything should link without any problems. Faster, but you don't learn about how, why, and all those other silly questions.

In [43]:
imp = bw.ExcelImporter(join("files", "excel_importer_example.xlsx"))
imp.apply_strategies()
imp.match_database(fields=('name', 'unit', 'location'))
imp.match_database("ecoinvent 2.2", fields=('name', 'unit', 'location'))
imp.statistics()

Extracted 3 worksheets in 0.01 seconds
Applying strategy: csv_restore_tuples
Applying strategy: csv_restore_booleans
Applying strategy: csv_numerize
Applying strategy: csv_drop_unknown
Applying strategy: normalize_units
Applying strategy: normalize_biosphere_categories
Applying strategy: normalize_biosphere_names
Applying strategy: strip_biosphere_exc_locations
Applying strategy: set_code_by_activity_hash
Applying strategy: link_iterable_by_fields
Applying strategy: assign_only_product_as_production
Applying strategy: link_technosphere_by_activity_hash
Applying strategy: drop_falsey_uncertainty_fields_but_keep_zeros
Applying strategy: convert_uncertainty_types_to_integers
Applied 14 strategies in 0.31 seconds
Applying strategy: link_iterable_by_fields
Applying strategy: link_iterable_by_fields
2 datasets
6 exchanges
0 unlinked exchanges
  


(2, 6, 0)

### Writing the database

Finally, we need to write this data to a new `Database`.

In [ ]:
imp.write_database()

Test to make sure we can do an LCA

In [ ]:
lca = bw.LCA(
    {("BW2 Excel water bottle import", "WriteSomeCode_UUID_isFineButNotNecessary"): 1}, 
    ('IPCC 2013', 'climate change', 'GWP 100a')
)
lca.lci()
lca.lcia()
lca.score

# Importing from ecospold 1

Please open the [following notebook](http://nbviewer.jupyter.org/urls/bitbucket.org/cmutel/brightway2/raw/default/notebooks/IO%20-%20Importing%20Agribalyse%20with%20Ecoinvent%202.2.ipynb), download it to this directory, and run it separately. For now, you can find a copy of [agribalyse 1.2 here](http://brightwaylca.org/data/agribalyse.zip); this will need to be unzipped in a child directory of the current directory.

# Importing from other formats

The procedure for importing from other formats is quite similar, you just have to choose the correct importer class. This does not mean that it is easy! Some formats are harder than others, but the [example notebooks](https://docs.brightwaylca.org/notebooks.html#example-notebooks) should at least point you in the right direction.